In [ ]:
import geemap
import ee
import numpy as np
from matplotlib.pyplot import cm

In [ ]:
ee.Initialize()

In [ ]:
Rd_Bu = ['#67001f','#b2182b','#d6604d','#f4a582','#fddbc7','#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061'][::-1]
Blues = ['#fff7fb','#ece7f2','#d0d1e6','#a6bddb','#74a9cf','#3690c0','#0570b0','#045a8d','#023858']
Yl_Bu = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#253494','#081d58']

In [ ]:
OFFSET_T = 273.15

In [ ]:
vis_params_t = {
  'min': -40 + OFFSET_T,
  'max': 40 + OFFSET_T,
  'palette': Rd_Bu}

In [ ]:
vis_params_trange = {
  'min': 0,
  'max': 40,
  'palette': Rd_Bu}

In [ ]:
vis_params_p = {
  'min': 0,
  'max': 0.4,
  'palette': Blues}

In [ ]:
def get_climate_month(collection, month_no, feature='mean_2m_air_temperature'):
    return collection.filter(ee.Filter.calendarRange(start=month_no, field='month')).select([feature]).mean()

In [ ]:
def calc_trange(image):
    band = image.expression('MAX - MIN',{'MAX': image.select('maximum_2m_air_temperature'),
                                         'MIN':image.select('minimum_2m_air_temperature')})
    return image.addBands(band.select([0], ['range_2m_air_temperature']))

## Load data 

In [ ]:
era = ee.ImageCollection("ECMWF/ERA5/MONTHLY")

In [ ]:
#SETTINGS
climate_period = ee.Filter.date('1981-01-01', '2010-12-31')

In [ ]:
era_climate = era.filter(climate_period).map(calc_trange)

## Make video animation 

### Climate 

In [ ]:
video_settings = {
  'dimensions': 768,
  'region': ee.Geometry.BBox(-180,-90, 180, 90),
  'framesPerSecond': 10}

#### Temperature 

In [ ]:
feature = 'mean_2m_air_temperature'
climate_collection_t = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
geemap.download_ee_video(climate_collection_t, {**video_settings,**vis_params_t}, 'ERA5_monthly_t.gif')

#### Dewpoint 

In [ ]:
feature='dewpoint_2m_temperature'
climate_collection_dp = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
geemap.download_ee_video(climate_collection_dp, {**video_settings,**vis_params_t}, 'ERA5_monthly_dp.gif')

#### Temperature range 

In [ ]:
feature='range_2m_air_temperature'
climate_collection_trange = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
geemap.download_ee_video(climate_collection_trange, {**video_settings,**vis_params_trange}, 'ERA5_monthly_trange.gif')

#### Precipitation 

In [ ]:
feature = 'total_precipitation'
climate_collection_p = ee.ImageCollection.fromImages([get_climate_month(era_climate, int(i), feature)for i in np.arange(1, 13)])

In [ ]:
geemap.download_ee_video(climate_collection_p, {**video_settings,**vis_params_p}, 'ERA5_monthly_p.gif')